# Hotel Chatbot

This Project is based on the Hotel Chatbot problem. How can we provide better customer service at the front desk of a hotel? One simple way is to have an FAQ chatbot that is capable of answering simple questions about the hotel experience. There are many advantages to having a chatbot. 

1. It increases the appeal of the hotel and also increases information throughput.
2. It creates a way to gather questions about the hotel in a data table form. 

In this notebook, I will be taking a look at 2 different models. The cosine similarity modela and the doc2vec model.


## 1. Adding knowledge base to your chatbot

The Chatbot's ability to converse is defined by the data available to it. Take a look at the Ques.txt file to find questions and ans.txt file to find answers to those questions. This chatbot will essentially run a cosine similarity on a question posed with the question bank and try to find an answer. 

Let us begin by importing relevant libraries.

In [1]:
import nltk # to process text data
import numpy as np # to represent corpus as arrays
import random 
import operator
import string # to process standard python strings
from sklearn.metrics.pairwise import cosine_similarity # We will use this later to decide how similar two sentences are
from sklearn.feature_extraction.text import TfidfVectorizer # Remember when you built a function to create a tfidf bag of words in Experience 2? This function does the same thing!

In [3]:
import os
filepath=os.getcwd()+r'\\hotelChat\\[Dataset] Module27 (ans).txt'
corpus=open(filepath,'r',errors = 'ignore')
raw_data_ans=corpus.read()
print (raw_data_ans)
filepath=os.getcwd()+'\\hotelChat\\[Dataset] Module27(ques).txt'
corpus=open(filepath,'r',errors = 'ignore')
raw_data=corpus.read()
print (raw_data)

200$ per night is the price for a basic suite.

This establishment was constructed and inaugurated by John S. on the 23rd of September 1965.

Breakfast is served from 7 AM to 10 AM.

The breakfast menu is decided as per the head chefs decision on the night before; kindly contact hotel staff for information about the menu on the night before. 

The Vance Hotel is a singular establishment designed and constructed by Lindsey Vance in 1949; it has hosted many dignitaries and government officials over the years.

There are 43 rooms in this hotel including one pent house suite.

We offer 4 types of rooms: basic, mid-level, premium and penthouse.

This hotel is called Vance Hotel.

Yes, room service is available 24 hrs.

To call room service, please dial '0' using the phone in your room.

Yes we have one restaurant currently called 'Rouge'.

There are 12 floors in the hotel.

300$ per night is the standard price for a mid-level suite.

500$ per night is the price for a premium suite.

Yes, we

####  Conversion to lower case

We will convert all text to lower case first. Remember to inspect the result once we are done.

In [4]:
raw_data=raw_data.lower()# converts to lowercase
print (raw_data)

what is the price of one night stay in basic suite?

how old is this establishment?

what time is breakfast served?

what is the breakfast menu?

what is the history behind this hotel or establishment?

how many rooms are there in this hotel?

what are the types of rooms or suites offered by the hotel?

what is the name of the hotel?

is room service served 24 hours?

how do i call for room service?

are there any restaurants in the hotel?

how many floors are there in the hotel?

what is the price of one night stay at the mid-level suite?

what is the price of one night stay at the premium suite?

do you have tuxedo services?

do you have a laundry service?

what time do the restaurants open for dinner?

what are the near by tourist attractions?

is there a spa in the hotel?

is there anywhere i can get a massage?

what time is the check in?

what time is the check out?

do you offer handicapped rooms?

is parking available at the hotel?

can i reserve a parking lot?

what are the rec

####  Segmentation, Lematization and tokenization

We will convert all text to lower case first. Remember to inspect the result once we are done.

In [5]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
sent_tokens = nltk.sent_tokenize(raw_data)# converts documents to list of sentences 

print(sent_tokens)

['what is the price of one night stay in basic suite?', 'how old is this establishment?', 'what time is breakfast served?', 'what is the breakfast menu?', 'what is the history behind this hotel or establishment?', 'how many rooms are there in this hotel?', 'what are the types of rooms or suites offered by the hotel?', 'what is the name of the hotel?', 'is room service served 24 hours?', 'how do i call for room service?', 'are there any restaurants in the hotel?', 'how many floors are there in the hotel?', 'what is the price of one night stay at the mid-level suite?', 'what is the price of one night stay at the premium suite?', 'do you have tuxedo services?', 'do you have a laundry service?', 'what time do the restaurants open for dinner?', 'what are the near by tourist attractions?', 'is there a spa in the hotel?', 'is there anywhere i can get a massage?', 'what time is the check in?', 'what time is the check out?', 'do you offer handicapped rooms?', 'is parking available at the hotel?

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\WIN10\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\WIN10\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
sent_tokens_ans = nltk.sent_tokenize(raw_data_ans)# converts documents to list of sentences 
print(sent_tokens_ans)

['200$ per night is the price for a basic suite.', 'This establishment was constructed and inaugurated by John S. on the 23rd of September 1965.', 'Breakfast is served from 7 AM to 10 AM.', 'The breakfast menu is decided as per the head chefs decision on the night before; kindly contact hotel staff for information about the menu on the night before.', 'The Vance Hotel is a singular establishment designed and constructed by Lindsey Vance in 1949; it has hosted many dignitaries and government officials over the years.', 'There are 43 rooms in this hotel including one pent house suite.', 'We offer 4 types of rooms: basic, mid-level, premium and penthouse.', 'This hotel is called Vance Hotel.', 'Yes, room service is available 24 hrs.', "To call room service, please dial '0' using the phone in your room.", "Yes we have one restaurant currently called 'Rouge'.", 'There are 12 floors in the hotel.', '300$ per night is the standard price for a mid-level suite.', '500$ per night is the price fo

In [7]:
res = {sent_tokens[i]: sent_tokens_ans[i] for i in range(len(sent_tokens))} 
print(res)

{'what is the price of one night stay in basic suite?': '200$ per night is the price for a basic suite.', 'how old is this establishment?': 'This establishment was constructed and inaugurated by John S. on the 23rd of September 1965.', 'what time is breakfast served?': 'Breakfast is served from 7 AM to 10 AM.', 'what is the breakfast menu?': 'The breakfast menu is decided as per the head chefs decision on the night before; kindly contact hotel staff for information about the menu on the night before.', 'what is the history behind this hotel or establishment?': 'The Vance Hotel is a singular establishment designed and constructed by Lindsey Vance in 1949; it has hosted many dignitaries and government officials over the years.', 'how many rooms are there in this hotel?': 'There are 43 rooms in this hotel including one pent house suite.', 'what are the types of rooms or suites offered by the hotel?': 'We offer 4 types of rooms: basic, mid-level, premium and penthouse.', 'what is the name 

In [8]:
word_tokens = nltk.word_tokenize(raw_data)# converts documents to list of words
print (word_tokens)

['what', 'is', 'the', 'price', 'of', 'one', 'night', 'stay', 'in', 'basic', 'suite', '?', 'how', 'old', 'is', 'this', 'establishment', '?', 'what', 'time', 'is', 'breakfast', 'served', '?', 'what', 'is', 'the', 'breakfast', 'menu', '?', 'what', 'is', 'the', 'history', 'behind', 'this', 'hotel', 'or', 'establishment', '?', 'how', 'many', 'rooms', 'are', 'there', 'in', 'this', 'hotel', '?', 'what', 'are', 'the', 'types', 'of', 'rooms', 'or', 'suites', 'offered', 'by', 'the', 'hotel', '?', 'what', 'is', 'the', 'name', 'of', 'the', 'hotel', '?', 'is', 'room', 'service', 'served', '24', 'hours', '?', 'how', 'do', 'i', 'call', 'for', 'room', 'service', '?', 'are', 'there', 'any', 'restaurants', 'in', 'the', 'hotel', '?', 'how', 'many', 'floors', 'are', 'there', 'in', 'the', 'hotel', '?', 'what', 'is', 'the', 'price', 'of', 'one', 'night', 'stay', 'at', 'the', 'mid-level', 'suite', '?', 'what', 'is', 'the', 'price', 'of', 'one', 'night', 'stay', 'at', 'the', 'premium', 'suite', '?', 'do', 'yo

In [9]:
lemmer = nltk.stem.WordNetLemmatizer() #Initiate lemmer class. WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

In [10]:
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict))) 

## 2. Adding Chatbot functionality - Cosine similarity

In [11]:
GREETING_INPUTS = ["hello", "hi", "greetings", "sup", "what's up","hey", "hey there"]
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

def greeting(sentence):
    for word in sentence.split(): # Looks at each word in your sentence
        if word.lower() in GREETING_INPUTS: # checks if the word matches a GREETING_INPUT
            return random.choice(GREETING_RESPONSES) # replies with a GREETING_RESPONSE

The functionality of the chatbot is done by creating a loop for running the chatbot. Take a loof at the function below. Each line in the function is important as it calls another function to perform an important step. The function 'response' is responsible for how the chatbot behaves. Having a master function for each functionality is recommended as good programming practise. 


In [12]:
def response(user_response):
    
    robo_response='' # initialize a variable to contain string
    sent_tokens.append(user_response) #add user response to sent_tokens
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english') 
    tfidf = TfidfVec.fit_transform(sent_tokens) #get tfidf value
    vals = cosine_similarity(tfidf[-1], tfidf) #get cosine similarity value
    idx=vals.argsort()[0][-2] 
    flat = vals.flatten() 
    flat.sort() #sort in ascending order
    req_tfidf = flat[-2] 
    
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response, vals
    else:
        robo_response = robo_response+res[sent_tokens[idx]]
        return robo_response, vals

Finally, let us create the chatbot interface and create a persona around it. Let us call it 'Jane' and use cosine similarity to find FAQs similar to the question asked and answer them. 

In [13]:

flag=True
print("Jane: My name is Jane. I will answer your queries about this hotel. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("Jane: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("Jane: "+greeting(user_response))

            else:
                print("Jane: ",end="")
                resp= response(user_response)
                print(resp[0], )
                sent_tokens.remove(user_response)
                resp_l = resp[1].tolist()
                resp_l[0].pop()
                print(' (With similarity of ',max(resp_l[0]),')')

    else:
        flag=False
        print("Jane: Bye! take care..")
       

Jane: My name is Jane. I will answer your queries about this hotel. If you want to exit, type Bye!
bye
Jane: Bye! take care..


by## 3. Chatbot functionality using Doc2vec

We will be covering one more type of model for creating chatbots. As we saw the cosine similarity model works on using that algorithm to find similarities between 2 sentences. But can we now use a neural network to solve this problem? Lets take a look.

Doc2Vec is a neural network based model that essentially creates vectors out of documents. In order to understand Doc2vec you also need to understand word2vec. 


#### What is word2vec?
It’s a Model to create the word embeddings, where it takes input as a large corpus of text and produces a vector space typically of several hundred dimesions. it was introduced in two papers between September and October 2013, by a team of researchers at Google. The underlying assumption of Word2Vec is that two words sharing similar contexts also share a similar meaning and consequently a similar vector representation from the model.

For instance: “Bank”, “money” and “accounts” are often used in similar situations, with similar surrounding words like “dollar”, “loan” or “credit”, and according to Word2Vec they will therefore share a similar vector representation. 

![Image](img1.png)

#### What is Doc2vec?

So the objective of Doc2vec is to create the numerical representation of sentence/paragraphs/documents unlike word2vec that computes a feature vector for every word in the corpus, Doc2Vec computes a feature vector for every document in the corpus. The vectors generated by Doc2vec can be used for tasks like finding similarity between sentences/paragraphs/documents

<strong> We will be using this property of doc2vec to create our own similarity model.</strong>


Let us begin by importing the relevant libraries.

In [14]:
import subprocess
import sys
subprocess.check_call([sys.executable, "-m", "pip", "install", "gensim == 3.8.1"])

0

In [15]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

The first main aim is to tag our data. The doc2vec model requires us to tag our data to use it effectively. Here is a good learning link for a [starter code](https://www.kaggle.com/fmitchell259/creating-a-doc2vec-model) for doc2vec. The following [link](https://medium.com/wisio/a-gentle-introduction-to-doc2vec-db3e8c0cce5e) can also be usedful and it recommended to read them. 

In [16]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(sent_tokens)]

In [17]:
max_epochs = 100
vec_size = 20 # Increase this to have a larger vector. This will mean more differentiation. 
alpha = 0.025


The next step is to train the model. As before we will be using a `model.train` function to run the training process. Take a look at the [documentation](https://radimrehurek.com/gensim/models/doc2vec.html) for more information about how to train the Doc2vec model. 

In [18]:

model = Doc2Vec(size =vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data, # tagged data to be used here
                total_examples=model.corpus_count,
                epochs=100)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")


D:\program\anaconda3\lib\site-packages\gensim\models\doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

### Evaluating doc2vec model

In [19]:
from gensim.models.doc2vec import Doc2Vec
model= Doc2Vec.load("d2v.model")

In [20]:
test_data = word_tokenize("How much is the price?".lower())

We can use the `model.infer_vector` function to infer the vector associated with a document. We can then use a function called `most_similar` to find the most similar vectors to the one we have created. What are the results? 

In [21]:
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

V1_infer [-0.31049836  0.06523347 -0.56080514 -0.08755983 -0.07490178 -0.27964514
 -0.17794505 -0.02328572 -0.06956264 -0.7482151   0.07187324  0.20198242
 -0.48601356 -0.24023212 -0.3307099   0.6302217  -0.31298116 -0.54201674
 -0.11918977  0.24365172]


In [22]:
similar_doc = model.docvecs.most_similar(positive = [v1], topn = 4) #positive is an attribute that shows positive correlation first followed by the correlation value
print(similar_doc)

[('41', 0.7254665493965149), ('25', 0.6682403087615967), ('27', 0.6634191870689392), ('38', 0.6337111592292786)]


In [23]:
num,_ = similar_doc[0]
num = int(num)
tagged_data[num]


TaggedDocument(words=['do', 'you', 'offer', 'hotel', 'shuttles', ',', 'for', 'example', ',', 'to', 'the', 'next', 'airport', 'or', 'train', 'station', '?'], tags=['41'])

Here it is very clear from the output of the previous code block that this model is not as effective as one might have thought. Are there any guesses why this model failed where we expected it to succeed?

Click on this [link](https://stackoverflow.com/questions/58206571/doc2vec-find-the-similar-sentence) to learn more about this. The gist of the issue is this

> Doc2Vec isn't going to give good results on toy-sized datasets, so you shouldn't expect anything meaningful until using much more data.

### Using pretrained doc2vec

And so let us use a pretrained model downloaded from the internet to optimize our use case. The model we have acquired is trained on the associated press news and can be downloaded [here](https://github.com/jhlau/doc2vec).

Let us load the model and evaluate it again.

In [27]:
model= Doc2Vec.load(os.getcwd()+r"\\hotelChat\\doc2vec.bin")

In [28]:
test_data = word_tokenize("How much is the price?".lower())

In [29]:
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

V1_infer [ 0.03475317 -0.0232788   0.00473981  0.00262373 -0.01251348  0.07123908
 -0.08395746 -0.06574792 -0.02189947  0.00627222  0.00682738  0.08678493
 -0.04159678 -0.0598289  -0.09617999  0.0339735  -0.0735061   0.03445594
 -0.06404027 -0.09197165 -0.09975794 -0.0393318   0.02397932  0.07204034
 -0.0021793  -0.10571146 -0.0257159  -0.0468796  -0.02843592  0.01477661
 -0.08576567 -0.00874451 -0.02925338  0.02389676  0.05420471  0.02479628
  0.01870595  0.00267763 -0.02965032 -0.10261499  0.02571831  0.02557221
  0.0552437   0.01456167 -0.02298301 -0.02992385  0.00195573 -0.05169348
 -0.01835014  0.05180986 -0.06865334 -0.04874812 -0.04033175 -0.03511868
  0.00690063 -0.05041308  0.01281186  0.03844377  0.04192817 -0.07401887
  0.06885412 -0.05911436 -0.05031609  0.05555575  0.02644834 -0.01291536
 -0.0963406  -0.08452649  0.09229217 -0.03283443 -0.03305859  0.05485052
 -0.02099306 -0.05570768  0.0150614   0.01698926  0.01828325  0.09762605
 -0.01140218  0.0387573  -0.0094849  -0.10

In [30]:
for i in sent_tokens:
    v2 = model.infer_vector(word_tokenize(i.lower()))
    print(i)
    print(cosine_similarity(v1.reshape(1, -1),v2.reshape(1, -1)))

what is the price of one night stay in basic suite?
[[0.5874423]]
how old is this establishment?
[[0.4858842]]
what time is breakfast served?
[[0.4509007]]
what is the breakfast menu?
[[0.5220691]]
what is the history behind this hotel or establishment?
[[0.42204374]]
how many rooms are there in this hotel?
[[0.5169172]]
what are the types of rooms or suites offered by the hotel?
[[0.4313618]]
what is the name of the hotel?
[[0.5501056]]
is room service served 24 hours?
[[0.33965442]]
how do i call for room service?
[[0.52169716]]
are there any restaurants in the hotel?
[[0.5188975]]
how many floors are there in the hotel?
[[0.4660408]]
what is the price of one night stay at the mid-level suite?
[[0.5426673]]
what is the price of one night stay at the premium suite?
[[0.5900573]]
do you have tuxedo services?
[[0.45683652]]
do you have a laundry service?
[[0.51981246]]
what time do the restaurants open for dinner?
[[0.5022719]]
what are the near by tourist attractions?
[[0.5370719]]
is 

In [31]:
def calc_prob(v1, q):
    probs = dict()
    for i in q:
        v2 = model.infer_vector(word_tokenize(i.lower()))
        sim = cosine_similarity(v1.reshape(1, -1),v2.reshape(1, -1))
        #print(i)
        #print(sim)
        probs[i] = sim[0][0]
    sorted_d = dict( sorted(probs.items(), key=operator.itemgetter(1),reverse=True))
    
    return list(sorted_d.items())[0]

In [32]:
calc_prob(v1, sent_tokens)

('why do the costs vary from day to day?', 0.6441365)

Since it appears that this model is more effective lets incorporate it into our chatbot.


In [33]:
flag=True
print("Jane: My name is Jane. I will answer your queries about this hotel. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("Jane: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("Jane: "+greeting(user_response))

            else:
                print("Jane: ",end="")
                resp= calc_prob(model.infer_vector(word_tokenize(user_response)), sent_tokens)
                print(res[resp[0]], )
                print(' (With similarity of ',resp[1],')')

    else:
        flag=False
        print("Jane: Bye! take care..")

Jane: My name is Jane. I will answer your queries about this hotel. If you want to exit, type Bye!
why do the costs vary from day to day?
Jane: The rates may vary as per availability and demand.
 (With similarity of  0.9145917 )
bye
Jane: Bye! take care..


**After observing the performance** of both of the models there are some clear conclusions that we can draw from this:

1. The Doc2vec model requires a lot more data for it to understand the relationship between words. And even after using a pre-trained model there is a clear lack of quality in the responses from the model compared to the cosine similarity model. 
2. The cosine similarity model works better for a smaller and more well defined dataset. This means that a few simple questions can be solved effectively but complex questions that require context wont be solved by it. 